In [1]:
import pandas as pd
import os
import requests
import json
import datetime
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# code for download the data
# specify path while downloading
data_apis_csv = {
 'cases': 'https://upcovid19tracks.in/api/get-cases?token=og33Y4KFKNqBaxnPkn44&csv=1',
 'lab_history': 'https://upcovid19tracks.in/api/get-lab-history?token=og33Y4KFKNqBaxnPkn44&csv=1'
}



def pull_data_from_api():
    '''Iterate thru all the apis and dump the data as csv files.'''
    script_dir = os.path.dirname(__file__) #<-- absolute dir the script is in
    print(datetime.datetime.now(), "Data Extraction starts....")
    for key, api in data_apis_csv.items():
        rel_path = key +".csv"
        abs_file_path = os.path.join(script_dir, rel_path)
        response = requests.get(api, allow_redirects=True)
        with open(abs_file_path, 'w', encoding='utf-8') as w:
            w.write(response.text)
    print(datetime.datetime.now(), "Data Extraction End....")

if __name__ == '__main__':
    # data dump
    pull_data_from_api()

In [2]:
#importing files 
case = pd.read_csv("cases.csv")
lab = pd.read_csv("lab_history.csv")
district = pd.read_excel("District masters.xlsx")

In [3]:
# merging case file ,lab_history file and district file
new_df = pd.merge(case,lab,left_on="case_id",right_on="case_id")
new_df = pd.merge(district[["ID","district_name"]],new_df,left_on="ID",right_on="district")

In [4]:
#converting track_date column to datetime format
new_df["track_date"] = pd.to_datetime(new_df["track_date"])

In [136]:
# saving merged file as csv
new_df.to_csv("merged_case.csv",index=True)

In [7]:
def confirmd(x):
    if x == 1:
        return 1
    else:
        return 0
new_df["confirmed-case"] = new_df.lab_result.apply(confirmd) 
new_df.to_csv("tableau_usecase.csv",index=True)

In [9]:
posetive_case = new_df[new_df.lab_result==1] #1 for +ve casese
posetive_case = posetive_case.groupby(["registration_date","district_name"])["case_id"].nunique().sort_values(ascending=False)[:15].reset_index()
posetive_case = posetive_case.rename(columns={"case_id":"total_case"})
posetive_case.to_csv("top_15_district.csv",index=True)#saving file for top 15 district total case
posetive_case

,registration_date,district_name,total_case
0,2020-04-14 19:38:14,AGRA,93
1,2020-04-14 19:38:14,GAUTAM BUDH NAGAR,63
2,2020-04-20 21:40:15,AGRA,59
3,2020-04-19 23:15:25,LUCKNOW,51
4,2020-04-19 23:15:25,AGRA,50
5,2020-04-14 19:38:14,Meerut,48
6,2020-04-14 19:38:14,LUCKNOW,36
7,2020-04-23 00:00:00,AGRA,32
8,2020-04-14 19:38:14,SAHARANPUR,30
9,2020-04-23 00:37:07,RAEBARELI,29


In [5]:
posetive_case = new_df[new_df.lab_result==1] #1 for +ve casese
posetive_case = posetive_case.groupby("district_name")["case_id"].nunique().sort_values(ascending=False)[:15].reset_index()
posetive_case = posetive_case.rename(columns={"case_id":"total_case"})
posetive_case.to_csv("top_15_district.csv",index=True)#saving file for top 15 district total case
posetive_case

,district_name,total_case
0,AGRA,439
1,KANPUR NAGAR,217
2,LUCKNOW,215
3,SAHARANPUR,186
4,GAUTAM BUDH NAGAR,138
5,FIROZABAD,113
6,MORADABAD,109
7,Meerut,103
8,GHAZIABAD,66
9,BULANDSHAHAR,54


In [22]:
# we have same gender different name so making it unique 
gender_dict = {"male":"Male","female":"Female","other":"Other","others":"Other"}
new_df["gender"] = new_df.gender.map(gender_dict)

# Q1. district wise active, recover, deceased and positive cases

In [26]:
posetive_case = new_df[new_df.lab_result==1] 
posetive_case = posetive_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df[(new_df.lab_result==1)&(new_df.status==1)]
active_case = active_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df[(new_df.lab_result==1)&(new_df.status==0)&(new_df.closed_reason==2)]
recover_case = recover_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df[(new_df.lab_result==1)&(new_df.status==0)&(new_df.closed_reason==4)]
deceased_case = deceased_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby(["district_name"])["positive","recover","active","deceased"].sum()
active_recover_deas_split[:5]
active_recover_deas_split.to_excel("tot_dist_PARD.xlsx",index=True)#saving file

,positive,recover,active,deceased
district_name,,,,
AGRA,419.0,32.0,374.0,13.0
AMROHA,27.0,0.0,27.0,0.0
AURAIYA,10.0,0.0,10.0,0.0
AYODHYA(FAIZABAD),1.0,0.0,1.0,0.0
AZAMGARH,8.0,3.0,5.0,0.0


# Q2. Daily trend for active, recovered, deceased case

In [29]:
posetive_case = new_df[new_df.lab_result==1] 
posetive_case = posetive_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df[(new_df.lab_result==1)&(new_df.status==1)]
active_case = active_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df[(new_df.lab_result==1)&(new_df.status==0)&(new_df.closed_reason==2)]
recover_case = recover_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df[(new_df.lab_result==1)&(new_df.status==0)&(new_df.closed_reason==4)]
deceased_case = deceased_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby(["track_date"])["positive","recover","active","deceased"].sum()
active_recover_deas_split[:5]
active_recover_deas_split.to_excel("daily_PARD_tot_dist.xlsx",index=True)# saving file

,positive,recover,active,deceased
track_date,,,,
2020-03-04,1.0,0.0,1.0,0.0
2020-03-06,1.0,0.0,1.0,0.0
2020-03-08,1.0,0.0,1.0,0.0
2020-03-13,1.0,0.0,1.0,0.0
2020-03-14,1.0,0.0,1.0,0.0


# Q3.Top10 district trend for PARD

In [31]:
new_df2 = new_df.copy()
new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby("track_date")["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby(["track_date"])["positive","recover","active","deceased"].sum()
active_recover_deas_split[:5]
active_recover_deas_split.to_excel("daily_PARD_focus_dist.xlsx",index=True)

,positive,recover,active,deceased
track_date,,,,
2020-03-04,1.0,0.0,1.0,0.0
2020-03-06,1.0,0.0,1.0,0.0
2020-03-13,1.0,0.0,1.0,0.0
2020-03-17,1.0,0.0,1.0,0.0
2020-03-18,3.0,1.0,2.0,0.0


# Q4. Top 10 distwise PARD cases

In [115]:
new_df2 = new_df.copy()
new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby("district_name")["positive","recover","active","deceased"].sum()
active_recover_deas_split[:5]
active_recover_deas_split.to_excel("PARD_focus_distwise.xlsx",index=True)

,positive,recover,active,deceased
district_name,,,,
AGRA,419.0,32.0,374.0,13.0
FIROZABAD,110.0,11.0,97.0,2.0
GAUTAM BUDH NAGAR,138.0,86.0,52.0,0.0
GHAZIABAD,63.0,30.0,33.0,0.0
KANPUR NAGAR,214.0,17.0,193.0,4.0


In [116]:
active_recover_deas_split.to_excel("PARD_focus_distwise.xlsx",index=True)

# Q5. focus dist recovery and deceased rate

In [63]:
new_df2 = new_df.copy()
new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]


total_case = new_df2[new_df2.lab_result==1]
total_case = total_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"total_case"})

active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})


recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([total_case,recover_case,deceased_case,active_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby("district_name")["recover","deceased","active","total_case"].sum()


#find out rate
active_recover_deas_split["recovery_rate"] = round((active_recover_deas_split["recover"]/active_recover_deas_split["total_case"]*100),0)

active_recover_deas_split["deceased_rate"] = round((active_recover_deas_split["deceased"]/active_recover_deas_split["total_case"]*100),0)

#active_recover_deas_split = active_recover_deas_split[["recovery_rate","deceased_rate","total_case"]]
active_recover_deas_split
active_recover_deas_split.to_excel("focus_dist_recover&deceased_rate.xlsx",index=True)

,recover,deceased,active,total_case,recovery_rate,deceased_rate
district_name,,,,,,
AGRA,32.0,13.0,374.0,419.0,8.0,3.0
FIROZABAD,11.0,2.0,97.0,110.0,10.0,2.0
GAUTAM BUDH NAGAR,86.0,0.0,52.0,138.0,62.0,0.0
GHAZIABAD,30.0,0.0,33.0,63.0,48.0,0.0
KANPUR NAGAR,17.0,4.0,193.0,214.0,8.0,2.0
LUCKNOW,35.0,1.0,172.0,208.0,17.0,0.0
MORADABAD,8.0,7.0,95.0,110.0,7.0,6.0
Meerut,49.0,5.0,48.0,102.0,48.0,5.0
SAHARANPUR,0.0,0.0,187.0,187.0,0.0,0.0


# Q6. focus district suspect cases

In [58]:
new_df2 = new_df[new_df.status==1]
new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

new_df2 = new_df2.groupby("district_name")["case_id"].nunique().sort_values(ascending=False).reset_index().rename(columns={"case_id":"suspect_case"})
#new_df2
new_df2.to_excel("focus_district_suspect_case.xlsx",index=False)

,district_name,suspect_case
0,KANPUR NAGAR,3055
1,SAHARANPUR,2885
2,LUCKNOW,2863
3,AGRA,2626
4,GHAZIABAD,2367
5,GAUTAM BUDH NAGAR,2223
6,Meerut,1791
7,MORADABAD,1401
8,VARANASI,1150
9,FIROZABAD,850


In [90]:
# taking care of missing value as it is not coming while running the code
new_df["gender"] = new_df.gender.fillna("NaN")
new_df["gender"] = new_df["gender"].replace({"Other":"others"})

# Q7. gender-wise split of PARD for focus district

In [93]:
new_df2 = new_df.copy()
new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby("Age_Group")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby("Age_Group")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby("Age_Group")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby("Age_Group")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby("Age_Group")["positive","recover","active","deceased"].sum()
#active_recover_deas_split[:5]
active_recover_deas_split.to_excel("focus_dist_genderARD.xlsx",index=True)

,positive,recover,active,deceased
gender,,,,
Female,207.0,15.0,190.0,2.0
Male,525.0,28.0,489.0,8.0
NaN,872.0,233.0,616.0,23.0


# Q8. gender-wise split of PARD for all district

In [130]:
new_df2 = new_df.copy()
#new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby("gender")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby("gender")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby("gender")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby("gender")["case_id"].nunique(dropna=False).reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby("gender")["positive","active","recover","deceased"].sum()
#active_recover_deas_split
active_recover_deas_split.to_excel("all_genderARD.xlsx",index=True)

,positive,active,recover,deceased
gender,,,,
Female,270.0,248.0,19.0,3.0
Male,638.0,589.0,39.0,10.0
NaN,1236.0,875.0,335.0,26.0


In [100]:
# function which can create a column for age_group
def Age_Group(x):
    if x >=0 and x <= 10:
        return "0-10"
    elif x >= 11 and x <= 20:
        return "10-20"
    elif x >= 21 and x <= 30:
        return "20-30"
    elif x >= 31 and x <= 40:
        return "30-40"
    elif x >= 41 and x <= 50:
        return "40-50"
    elif x >= 51 and x <= 60:
        return "50-60"
    elif x >= 61 and x <= 70:
        return "60-70"
    elif x >= 71 and x <= 80:
        return "70-80"
    elif x >= 81 and x <= 90:
        return "80-90"
    elif x >= 91 and x <= 191:
        return "90 above"
    else:
        return "Not_Present"
# applying above function to age column
new_df["Age_Group"] = new_df["age"].apply(Age_Group)

# Q9. all district agegroup wise PARD cases

In [120]:
new_df2 = new_df.copy()
#new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby("Age_Group")["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby("Age_Group")["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby("Age_Group")["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby("Age_Group")["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby("Age_Group")["positive","active","recover","deceased"].sum()
#active_recover_deas_split
active_recover_deas_split.to_excel("All_agegroup_PARD.xlsx",index=True)

,positive,active,recover,deceased
Age_Group,,,,
0-10,67.0,55.0,12.0,0.0
10-20,351.0,276.0,75.0,0.0
20-30,630.0,536.0,93.0,1.0
30-40,395.0,313.0,79.0,3.0
40-50,300.0,239.0,55.0,6.0
50-60,215.0,163.0,40.0,12.0
60-70,126.0,90.0,25.0,11.0
70-80,43.0,27.0,10.0,6.0
80-90,4.0,2.0,2.0,0.0


# Q10. gender and agegroup-wise PARD cases for all district

In [122]:
new_df2 = new_df.copy()
#new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby(["gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby(["gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby(["gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby(["gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby(["gender","Age_Group"])["positive","active","recover","deceased"].sum()
#active_recover_deas_split
active_recover_deas_split.to_excel("All_gender_agegroup.xlsx",index=True)

positive  active  recover  deceased
gender Age_Group                                       
Female 0-10             16.0    15.0      1.0       0.0
       10-20            50.0    45.0      5.0       0.0
       20-30            72.0    69.0      3.0       0.0
       30-40            54.0    50.0      4.0       0.0
       40-50            35.0    31.0      3.0       1.0
       50-60            27.0    25.0      1.0       1.0
       60-70             9.0     8.0      0.0       1.0
       70-80             5.0     3.0      2.0       0.0
       80-90             2.0     2.0      0.0       0.0
Male   0-10             10.0    10.0      0.0       0.0
       10-20            92.0    86.0      6.0       0.0
       20-30           203.0   192.0     11.0       0.0
       30-40           123.0   118.0      4.0       1.0
       40-50            93.0    88.0      5.0       0.0
       50-60            68.0    56.0     10.0       2.0
       60-70            39.0    30.0      3.0       6.0
       70-80            10.0     9.0      0.0       1.0
NaN    0-10             41.0    30.0     11.0       0.0
       10-20           209.0   145.0     64.0       0.0
       20-30           355.0   275.0     79.0       1.0
       30-40           218.0   145.0     71.0       2.0
       40-50           172.0   120.0     47.0       5.0
       50-60           120.0    82.0     29.0       9.0
       60-70            78.0    52.0     22.0       4.0
       70-80            28.0    15.0      8.0       5.0
       80-90             2.0     0.0      2.0       0.0
       Not_Present      13.0    11.0      2.0       0.0

# Q11. all district and agegroup-wise PARD cases

In [124]:
new_df2 = new_df.copy()
#new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby(["district_name","gender"])["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby(["district_name","gender"])["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby(["district_name","gender"])["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby(["district_name","gender"])["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby(["district_name","gender"])["positive","active","recover","deceased"].sum()
#active_recover_deas_split
active_recover_deas_split.to_excel("All_disname-gender.xlsx",index=True)

positive  active  recover  deceased
district_name    gender                                     
AGRA             Female      57.0    51.0      5.0       1.0
                 Male       156.0   136.0     14.0       6.0
                 NaN        206.0   187.0     13.0       6.0
AMROHA           Male         4.0     4.0      0.0       0.0
                 NaN         23.0    23.0      0.0       0.0
...                           ...     ...      ...       ...
Sant Kabir Nagar NaN         10.0    10.0      0.0       0.0
UNNAV            NaN          1.0     1.0      0.0       0.0
VARANASI         Female       5.0     5.0      0.0       0.0
                 Male        30.0    30.0      0.0       0.0
                 NaN         18.0     9.0      8.0       1.0

[126 rows x 4 columns]

# Q12. All district district name and agegroup-wise PARD cases

In [126]:
new_df2 = new_df.copy()
#new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby(["district_name","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby(["district_name","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby(["district_name","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby(["district_name","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby(["district_name","Age_Group"])["positive","active","recover","deceased"].sum()
#active_recover_deas_split
active_recover_deas_split.to_excel("All_disname-AgeGroup.xlsx",index=True)

positive  active  recover  deceased
district_name Age_Group                                       
AGRA          0-10              8.0     8.0      0.0       0.0
              10-20            42.0    36.0      6.0       0.0
              20-30           136.0   128.0      8.0       0.0
              30-40            91.0    86.0      4.0       1.0
              40-50            70.0    65.0      5.0       0.0
...                             ...     ...      ...       ...
VARANASI      40-50            11.0    10.0      1.0       0.0
              50-60             9.0     7.0      1.0       1.0
              60-70             1.0     1.0      0.0       0.0
              70-80             1.0     1.0      0.0       0.0
              Not_Present       1.0     1.0      0.0       0.0

[282 rows x 4 columns]

# Q13. All district dist, gendder and age-wise PARD cases

In [128]:
new_df2 = new_df.copy()
#new_df2 = new_df2[(new_df2.district_name=="AGRA")|(new_df2.district_name=="LUCKNOW")|(new_df2.district_name=="KANPUR NAGAR")|(new_df2.district_name=="SAHARANPUR")|(new_df2.district_name=="GAUTAM BUDH NAGAR")|(new_df2.district_name=="MORADABAD")|(new_df2.district_name=="FIROZABAD")|(new_df2.district_name=="Meerut")|(new_df2.district_name=="GHAZIABAD")|(new_df2.district_name=="VARANASI")]

posetive_case = new_df2[new_df2.lab_result==1] 
posetive_case = posetive_case.groupby(["district_name","gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"positive"})


active_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==1)]
active_case = active_case.groupby(["district_name","gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"active"})

recover_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==2)]
recover_case = recover_case.groupby(["district_name","gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"recover"})

deceased_case = new_df2[(new_df2.lab_result==1)&(new_df2.status==0)&(new_df2.closed_reason==4)]
deceased_case = deceased_case.groupby(["district_name","gender","Age_Group"])["case_id"].nunique().reset_index().rename(columns={"case_id":"deceased"})

active_recover_deas_split = pd.concat([posetive_case,active_case,recover_case,deceased_case],axis=0)
active_recover_deas_split = active_recover_deas_split.fillna(0)
active_recover_deas_split = active_recover_deas_split.groupby(["district_name","gender","Age_Group"])["positive","active","recover","deceased"].sum()
#active_recover_deas_split
active_recover_deas_split.to_excel("All_disname-AgeGroup_gender.xlsx",index=True)

positive  active  recover  deceased
district_name gender Age_Group                                       
AGRA          Female 0-10              2.0     2.0      0.0       0.0
                     10-20             6.0     4.0      2.0       0.0
                     20-30            17.0    17.0      0.0       0.0
                     30-40            16.0    16.0      0.0       0.0
                     40-50            10.0     9.0      1.0       0.0
...                                    ...     ...      ...       ...
VARANASI      NaN    30-40             3.0     1.0      2.0       0.0
                     40-50             2.0     1.0      1.0       0.0
                     50-60             3.0     1.0      1.0       1.0
                     70-80             1.0     1.0      0.0       0.0
                     Not_Present       1.0     1.0      0.0       0.0

[477 rows x 4 columns]

# Q13. All district-wise total no of cases

In [117]:
new_df2 = new_df.copy()
total_case = new_df2[new_df2.lab_result==1]
total_case = total_case.groupby("district_name")["case_id"].nunique().reset_index().rename(columns={"case_id":"total_case"})

print("Total case all over the district:",total_case.total_case.sum())

# ========================END==============================